## import stuff 

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 


CV = True 

pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-broadbandpoly2/output/desi_kp4_abacus_cubic_ELG-broadbandpoly2.fitter.pkl"

pfn2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-polybroadband2/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree-polybroadband2.fitter.pkl"
    
pfns = [
        pfn1, 
        pfn2
]



## looping through and saving the chains 

In [ ]:
c = ChainConsumer() 

for i,p in enumerate(pfns): 
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']:
            continue 
            
        if 'Pk' not in extra['name']:
            continue 
#         CVfile = '' # making sure we adjust this parameter if CV or not and skipping depending on value of CV 
#         if CV:
#             CVfile = 'CV'
#             if "CV" not in extra["name"] or "Pk" not in extra["name"]:
#                continue
#         else:
#             if "CV" in extra['name']:
#                 continue 
#             if "Pk" not in extra['name']:
#                 continue 
                
                
        df = pd.DataFrame(chain, columns=model.get_labels())
    
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp
        
        extraname = ''
        if i == 1:
            extraname = ' beta model'
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extra.pop("realisation", 'mean')#+ extraname)
        extra["name"] = extra['name'] + extraname
        
        #print(extra)

        c.add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        #print('test')
        

In [ ]:
truth = {"$\\alpha$": 1.0, "$\\epsilon$": 0, "$\\alpha_\\perp$": 1.0, "$\\alpha_\\parallel$": 1.0}
#truth = {"$\\alpha_\\parallel$": 1.0, "$\\alpha_\\perp$": 1.0, "$\\alpha_\\perp$": 1.0, "$\\alpha_\\parallel$": 1.0}


truth["$\\beta_{\\phi(N_{\\mathrm{eff}})}$"] = 1.0

parameters = [names for names in truth]
parameters.append("$\\Sigma_{nl,||}$")
parameters.append("$\\Sigma_{nl,\\perp}$")
parameters.append("$\\Sigma_s$")

print(dir(c))

c.plotter.plot(
    truth=truth,
        parameters = parameters[:2] + [parameters[4]],
        legend=True,
        display=False,
        figsize=(12,12), 
    )

## import stuff (now plotting with varying Omega_m, Neff) 

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 


CV = True 

pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_omfree/output/desi_kp4_abacus_cubic_ELG_omfree.fitter.pkl"

pfn2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_nefffree/output/desi_kp4_abacus_cubic_ELG_nefffree.fitter.pkl"
    
pfns = [
        pfn1, 
        pfn2
]


## do the loops 

In [ ]:
c = ChainConsumer() 

for i,p in enumerate(pfns): 
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']:
            continue 
            
        # if 'Pk' not in extra['name']:
        #     continue 
#         CVfile = '' # making sure we adjust this parameter if CV or not and skipping depending on value of CV 
#         if CV:
#             CVfile = 'CV'
#             if "CV" not in extra["name"] or "Pk" not in extra["name"]:
#                continue
#         else:
#             if "CV" in extra['name']:
#                 continue 
#             if "Pk" not in extra['name']:
#                 continue 
                
                
        df = pd.DataFrame(chain, columns=model.get_labels())
    
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp
        
        extraname = ''
        if i == 0:
            extraname = ' om model'
        if i == 1: 
            extraname = ' neff model'
            
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extra.pop("realisation", 'mean')#+ extraname)
        extra["name"] = extra['name'] + extraname
        
        #print(extra)
        #print(df)

        c.add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        #print('test')
        

In [ ]:
c = ChainConsumer() 

for i,p in enumerate(pfns): 
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']:
            continue 
            
        # if 'Pk' not in extra['name']:
        #     continue 
#         CVfile = '' # making sure we adjust this parameter if CV or not and skipping depending on value of CV 
#         if CV:
#             CVfile = 'CV'
#             if "CV" not in extra["name"] or "Pk" not in extra["name"]:
#                continue
#         else:
#             if "CV" in extra['name']:
#                 continue 
#             if "Pk" not in extra['name']:
#                 continue 
                
                
        df = pd.DataFrame(chain, columns=model.get_labels())
    
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp
        
        extraname = ''
        if i == 0:
            extraname = ' om model'
        if i == 1: 
            extraname = ' neff model'
            
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extra.pop("realisation", 'mean')#+ extraname)
        extra["name"] = extra['name'] + extraname
        
        #print(extra)
        #print(df)

        c.add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        #print('test')
        

In [ ]:
truth = {"$\\alpha$": 1.0, "$\\epsilon$": 0, "$\\alpha_\\perp$": 1.0, "$\\alpha_\\parallel$": 1.0}

truth["$\\Omega_m$"] = 0.31
truth["$N_{\\mathrm{eff}}$"] = 3.044

parameters = [names for names in truth]
# parameters.append("$\\Sigma_{nl,||}$")
# parameters.append("$\\Sigma_{nl,\\perp}$")
# parameters.append("$\\Sigma_s$")

print(dir(c))

c.plotter.plot(
    truth=truth,
        parameters = parameters[:2]+parameters[4:],
        legend=True,
        display=False,
        figsize=(9,9), 
    )

## attempt 3 - best nuisance parameters 

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 

pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_bestnuisance/output/desi_kp4_abacus_cubic_ELG_bestnuisance.fitter.pkl"
 
pfns = [
        pfn1, 
]


In [ ]:
c = ChainConsumer() 

for i,p in enumerate(pfns): 
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']:
            continue 
            
        # if 'Pk' not in extra['name']:
        #     continue 
#         CVfile = '' # making sure we adjust this parameter if CV or not and skipping depending on value of CV 
#         if CV:
#             CVfile = 'CV'
#             if "CV" not in extra["name"] or "Pk" not in extra["name"]:
#                continue
#         else:
#             if "CV" in extra['name']:
#                 continue 
#             if "Pk" not in extra['name']:
#                 continue 
                
                
        df = pd.DataFrame(chain, columns=model.get_labels())
    
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extra.pop("realisation", 'mean')#+ extraname)
        #extra["name"] = extra['name'] + extraname
        
        #print(extra)
        #print(df)

        c.add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        #print('test')

In [ ]:
#truth = {"$\\alpha$": 1.0, "$\\epsilon$": 0, "$\\alpha_\\perp$": 1.0, "$\\alpha_\\parallel$": 1.0}

#truth["$\\Omega_m$"] = 0.31
#truth["$N_{\\mathrm{eff}}$"] = 3.044

#parameters = [names for names in truth]
# parameters.append("$\\Sigma_{nl,||}$")
# parameters.append("$\\Sigma_{nl,\\perp}$")
# parameters.append("$\\Sigma_s$")

print(dir(c))

c.plotter.plot(
    #truth=truth,
        parameters = ["$\\Sigma_s$", "$\\Sigma_{nl,\\perp}$", "$\Sigma_{nl,||}$"],
        legend=True,
        display=False,
        figsize=(9,9), 
    )

## now looking at fits to 6 c003 cubic mocks with c000 clustering power spectrum measurements (template is c000)

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 

pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-c003/output/desi_kp4_abacus_cubic_ELG-c003.fitter.pkl"

pfn2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003.fitter.pkl"
    
pfns = [
        pfn1, 
        pfn2
]


c = ChainConsumer() 

for i,p in enumerate(pfns): 
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']: # making sure we only get results from recon 
            continue  
                
        df = pd.DataFrame(chain, columns=model.get_labels())
        
        print(df)
    
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp
        
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extraname = ''
        if i == 1:
            extraname = 'phaseshift'
            
        extra.pop("realisation", 'mean' + extraname)
        extra["name"] = extra['name'] + extraname
        
        #print(extra)
        #print(df)

        c.add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        #print('test')
    
couplingterm = 1.0/0.22710731766023898
beta_approx = (3.044/(3.044+couplingterm) * (3.7+couplingterm)/3.7) 

alpha, eps = model.get_reverse_alphas(0.99906947, 0.98927880) 

truth = {"$\\alpha$": alpha, "$\\epsilon$": eps, "$\\alpha_\\parallel$": 0.99906947, "$\\alpha_\\perp$": 0.98927880, 
         "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": beta_approx}

parameters = [names for names in truth]

c.plotter.plot(
    truth=truth,
        parameters = parameters[:2]+parameters[4:], 
        legend=True,
        display=False,
        figsize=(9,9), 
)

## now looking at fits to 6 c003 cubic mocks with c000 clustering power spectrum measurements (template is c003!)

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 

pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-c003-template003/output/desi_kp4_abacus_cubic_ELG-c003-template003.fitter.pkl"

pfn2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-template003/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-template003.fitter.pkl"
    
pfns = [
        pfn1, 
        pfn2
]


c = ChainConsumer() 

for i,p in enumerate(pfns): 
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']: # making sure we only get results from recon 
            continue  
                
        df = pd.DataFrame(chain, columns=model.get_labels())
        
        print(df)
    
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp
        
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extraname = ''
        if i == 1:
            extraname = ' phaseshift'
            
        extra.pop("realisation", 'mean' + extraname)
        extra["name"] = extra['name'] + extraname
        
        #print(extra)
        #print(df)

        c.add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        #print('test')
    
#couplingterm = 1.0/0.22710731766023898
#beta_approx = (3.044/(3.044+couplingterm) * (3.7+couplingterm)/3.7) 

alpha, eps = model.get_reverse_alphas(1.02131937, 1.01131066) 

truth = {"$\\alpha$": alpha, "$\\epsilon$": eps, "$\\alpha_\\parallel$": 1.02131937, "$\\alpha_\\perp$": 1.01131066, 
         "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": 1.0}

parameters = [names for names in truth]

c.plotter.plot(
    truth=truth,
        parameters = parameters[:2]+parameters[4:], 
        legend=True,
        display=False,
        figsize=(9,9), 
)

## comparing smoothing methods results for phase shift constraints for c000 ELG mocks with c000 clustering 

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 


pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree.fitter.pkl"

pfn2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-wallisch18/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree-wallisch18.fitter.pkl"

pfn3 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-eh1998/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree-eh1998.fitter.pkl"

pfns = [
    
        pfn1, 
        pfn2,
        pfn3
]


c = ChainConsumer() 

fig, axes = plt.subplots(3, 2, sharex=True, sharey=True)

#ks = np.logspace(-3, 2, 100, base=10)

for i,p in enumerate(pfns): 
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    count = 0
    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']: # making sure we only get results from recon 
            continue  
            
        # if 'CV' in extra['name']:
        #     continue
            
        if 'Xi' in extra['name']:
            continue
                
        df = pd.DataFrame(chain, columns=model.get_labels())
    
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp
        
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extraname = ''
        if i == 0:
            extraname = ' Hinton17'
        elif i == 1:
            extraname = ' Wallisch18'
        elif i == 2:
            extraname = ' eh1998'
            
        if 'CV' in extra['name']:
            extraname = extraname + ' CV'
            
        extra.pop("realisation", 'mean' + extraname)
        extra["name"] = extra['name'] + extraname
       
        c.add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        
        model.set_data(data)
        r_s = model.camb.get_data()["r_s"]
        max_post = posterior.argmax()
        params = df.loc[max_post]
        params_dict = model.get_param_dict(chain[max_post])
        for name, val in params_dict.items():
            model.set_default(name, val)

        new_chi_squared, dof, bband, mods, smooths = model.plot(params_dict, display=False)
        k = model.data[0]["ks"]
        axes[i][count].scatter(k, (model.data[0]['pk0'][0]- smooths[0][0])*k, label= extraname + ' data')
        axes[i][count].plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        axes[i][count].legend()
        count += 1
        
        
plt.subplots_adjust(wspace=0, hspace=0)        
        
        
#couplingterm = 1.0/0.22710731766023898
#beta_approx = (3.044/(3.044+couplingterm) * (3.7+couplingterm)/3.
#alpha, eps = model.get_reverse_alphas(1.02131937, 1.01131066) 



truth = {"$\\alpha$": 1.0, "$\\epsilon$": 0.0, "$\\alpha_\\parallel$": 1, "$\\alpha_\\perp$": 1,
         "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": 1.0}

parameters = [names for names in truth]

plt.legend()
plt.xlabel(r'$k$ $\mathrm{Mpc}^{-1} h$')
plt.ylabel(r'$k P_0(k)$ $\mathrm{Mpc}^2 h^{-2}$')

c.plotter.plot(
    truth=truth,
        parameters = parameters[:2]+parameters[4:], 
        legend=True,
        display=False,
        figsize=(11, 11), 
)

## comparing smoothing methods results for phase shift constraints for c003 ELG mocks with c000 clustering, c000 template

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 


pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003.fitter.pkl"

pfn2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-Wallisch/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-Wallisch.fitter.pkl"

pfn3 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-eh1998/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-eh1998.fitter.pkl"


# pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-template003/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-template003.fitter.pkl"

# pfn2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-template003-Wallisch/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-template003-Wallisch.fitter.pkl"

# pfn3 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-template003-eh1998/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-template003-eh1998.fitter.pkl"

pfns = [
        pfn1, 
        pfn2,
        pfn3
]

c = ChainConsumer() 

for i,p in enumerate(pfns): 
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']: # making sure we only get results from recon 
            continue  
            
        #if 'CV' not in extra['name']:
        #    continue
                
        df = pd.DataFrame(chain, columns=model.get_labels())
        
        #print(df)
    
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp
        
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extraname = ''
        if i == 0:
            extraname = ' Hinton17'
        elif i == 1:
            extraname = ' Wallisch18'
        elif i == 2:
            extraname = ' eh1998'
            
        extra.pop("realisation", 'mean' + extraname)
        extra["name"] = extra['name'] + extraname
        
        #print(extra)
        #print(df)

        c.add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        #print('test')
        
        model.set_data(data)
        r_s = model.camb.get_data()["r_s"]
        max_post = posterior.argmax()
        params = df.loc[max_post]
        params_dict = model.get_param_dict(chain[max_post])
        for name, val in params_dict.items():
            model.set_default(name, val)

        new_chi_squared, dof, bband, mods, smooths = model.plot(params_dict, display=False)
        k = model.data[0]["ks"]
        #plt.scatter(k, model.data[0]['pk0'][0]- smooths[0][0], label= extraname + ' data')
        plt.plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        
    
couplingterm = 1.0/0.22710731766023898
beta_approx = (3.044/(3.044+couplingterm) * (3.7+couplingterm)/3.7) 

alpha, eps = model.get_reverse_alphas(0.99906947, 0.98927880) 

truth = {"$\\alpha$": alpha, "$\\epsilon$": eps, "$\\alpha_\\parallel$": 0.99906947, "$\\alpha_\\perp$": 0.98927880, 
         "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": beta_approx}


# alpha, eps = model.get_reverse_alphas(1.02131937, 1.01131066) 

# truth = {"$\\alpha$": alpha, "$\\epsilon$": eps, "$\\alpha_\\parallel$": 1.02131937, "$\\alpha_\\perp$": 1.01131066, 
#          "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": 1.0}


parameters = [names for names in truth]

c.plotter.plot(
    truth=truth,
        parameters = parameters[:2]+parameters[4:],#+["$\\beta$"], 
        legend=True,
        display=False,
        figsize=(12, 12), 
)

## plot results with Hinton dewiggling - different broadband settings, c000/grid000 mocks - power spectrum only 

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 


pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG/output/desi_kp4_abacus_cubic_ELG.fitter.pkl"

# pfn2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_splinebroadband/output/desi_kp4_abacus_cubic_ELG_splinebroadband.fitter.pkl"

pfn3 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-broadbandpoly/output/desi_kp4_abacus_cubic_ELG-broadbandpoly.fitter.pkl"

pfns = [
    
        pfn1, 
        #pfn2,
        pfn3
]


c = ChainConsumer() 

# fig, axes = plt.subplots(3, 2, sharex=True, sharey=True)

# ks = np.logspace(-3, 2, 100, base=10)

for i,p in enumerate(pfns): 
    #print(i)
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    count = 0
    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']: # making sure we only get results from recon 
            continue  
            
        # if 'CV' in extra['name']:
        #     continue
            
        if 'Xi' in extra['name']:
            continue
                
        df = pd.DataFrame(chain, columns=model.get_labels())
    
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp
        
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extraname = ''
        if i == 0:
            extraname = ' None'
        elif i == 1:
            extraname = ' poly'
        #elif i == 2:
         #   extraname = ' poly'
         #   
        if 'CV' in extra['name']:
            extraname = extraname + ' CV'
            
        extra.pop("realisation", 'mean' + extraname)
        extra["name"] = extra['name'] + extraname
       
        c.add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        
        model.set_data(data)
        r_s = model.camb.get_data()["r_s"]
        max_post = posterior.argmax()
        params = df.loc[max_post]
        params_dict = model.get_param_dict(chain[max_post])
        for name, val in params_dict.items():
            model.set_default(name, val)

        new_chi_squared, dof, bband, mods, smooths = model.plot(params_dict, display=False)
        k = model.data[0]["ks"]
        # axes[i][count].scatter(k, (model.data[0]['pk0'][0]- smooths[0][0])*k, label= extraname + ' data')
        # axes[i][count].plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # axes[i][count].legend()
        
        #plt.scatter(k, (model.data[0]['pk0'][0]- smooths[0][0])*k, label= extraname + ' data')
        plt.plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        plt.legend()
        count += 1

        
        # if i == 0:
        #     default = smooths[0][0]
        # # plt.plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # plt.plot(k, (smooths[0][0]/default)*k, label=extraname)
        
        
        
# plt.subplots_adjust(wspace=0, hspace=0)        
        
        
#couplingterm = 1.0/0.22710731766023898
#beta_approx = (3.044/(3.044+couplingterm) * (3.7+couplingterm)/3.
#alpha, eps = model.get_reverse_alphas(1.02131937, 1.01131066) 



truth = {"$\\alpha$": 1.0, "$\\epsilon$": 0.0, "$\\alpha_\\parallel$": 1, "$\\alpha_\\perp$": 1,
        }# "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": 1.0}

parameters = [names for names in truth]

# plt.legend()
# plt.xlabel(r'$k$ $\mathrm{Mpc}^{-1} h$')
# plt.ylabel(r'$k P_0(k)$ $\mathrm{Mpc}^2 h^{-2}$')

c.plotter.plot(
    truth=truth,
        parameters = parameters[:2]+parameters[4:], 
        legend=True,
        display=False,
        figsize=(11, 11), 
)

## plot results with Hinton dewiggling - different broadband settings, c003/grid000/template000/003 mocks - power spectrum only 

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 


pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-c003/output/desi_kp4_abacus_cubic_ELG-c003.fitter.pkl"

#pfn2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-splinebroadband/output/desi_kp4_abacus_cubic_ELG_phaseshiftfree-splinebroadband.fitter.pkl"

pfn3 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-c003-broadbandpoly/output/desi_kp4_abacus_cubic_ELG-c003-broadbandpoly.fitter.pkl"

pfns = [
    
        pfn1, 
        #pfn2,
        pfn3
]


c = ChainConsumer() 

# fig, axes = plt.subplots(3, 2, sharex=True, sharey=True)

# ks = np.logspace(-3, 2, 100, base=10)

for i,p in enumerate(pfns): 
    #print(i)
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    count = 0
    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']: # making sure we only get results from recon 
            continue  
            
        # if 'CV' in extra['name']:
        #     continue
            
        if 'Xi' in extra['name']:
            continue
                
        df = pd.DataFrame(chain, columns=model.get_labels())
    
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp
        
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extraname = ''
        if i == 0:
            extraname = ' None'
        elif i == 1:
            extraname = ' poly'
#         elif i == 2:
#             extraname = ' poly'
            
        if 'CV' in extra['name']:
            extraname = extraname + ' CV'
            
        extra.pop("realisation", 'mean' + extraname)
        extra["name"] = extra['name'] + extraname
       
        c.add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        
        model.set_data(data)
        r_s = model.camb.get_data()["r_s"]
        max_post = posterior.argmax()
        params = df.loc[max_post]
        params_dict = model.get_param_dict(chain[max_post])
        for name, val in params_dict.items():
            model.set_default(name, val)

        new_chi_squared, dof, bband, mods, smooths = model.plot(params_dict, display=False)
        k = model.data[0]["ks"]
        # axes[i][count].scatter(k, (model.data[0]['pk0'][0]- smooths[0][0])*k, label= extraname + ' data')
        # axes[i][count].plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # axes[i][count].legend()
        
        #plt.scatter(k, (model.data[0]['pk0'][0]- smooths[0][0])*k, label= extraname + ' data')
        plt.plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        plt.legend()
        count += 1

        
        # if i == 0:
        #     default = smooths[0][0]
        # # plt.plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # plt.plot(k, (smooths[0][0]/default)*k, label=extraname)
        
        
        
# plt.subplots_adjust(wspace=0, hspace=0)        
        
        
couplingterm = 1.0/0.22710731766023898
beta_approx = (3.044/(3.044+couplingterm) * (3.7+couplingterm)/3.7) 

alpha, eps = model.get_reverse_alphas(0.99906947, 0.98927880) 

truth = {"$\\alpha$": alpha, "$\\epsilon$": eps, "$\\alpha_\\parallel$": 0.99906947, "$\\alpha_\\perp$": 0.98927880, 
        }#"$\\beta_{\\phi(N_{\\mathrm{eff}})}$": beta_approx}

# alpha, eps = model.get_reverse_alphas(1.02131937, 1.01131066) 

# truth = {"$\\alpha$": alpha, "$\\epsilon$": eps, "$\\alpha_\\parallel$": 1.02131937, "$\\alpha_\\perp$": 1.01131066, 
#         }# "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": 1.0}

parameters = [names for names in truth]

# plt.legend()
# plt.xlabel(r'$k$ $\mathrm{Mpc}^{-1} h$')
# plt.ylabel(r'$k P_0(k)$ $\mathrm{Mpc}^2 h^{-2}$')

c.plotter.plot(
    truth=truth,
        parameters = parameters[:2]+parameters[4:], 
        legend=True,
        display=False,
        figsize=(11, 11), 
)

## plot results with Hinton dewiggling - broadband_type = poly, c000/grid003/template000/003 mocks - power spectrum only 

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 


pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-c000_grid003-template000/output/desi_kp4_abacus_cubic_ELG-c000_grid003-template000.fitter.pkl"

pfn2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-c000_grid003-template000-phaseshift/output/desi_kp4_abacus_cubic_ELG-c000_grid003-template000-phaseshift.fitter.pkl"

pfn3 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-c000_grid003-template003/output/desi_kp4_abacus_cubic_ELG-c000_grid003-template003.fitter.pkl"

pfn4 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-c000_grid003-template003-phaseshift/output/desi_kp4_abacus_cubic_ELG-c000_grid003-template003-phaseshift.fitter.pkl"


pfns = [
    
        # pfn1, 
        # pfn2,
        pfn3,
        pfn4, 
]


c = ChainConsumer() 

# fig, axes = plt.subplots(3, 2, sharex=True, sharey=True)

# ks = np.logspace(-3, 2, 100, base=10)

for i,p in enumerate(pfns): 
    #print(i)
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    count = 0
    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']: # making sure we only get results from recon 
            continue  
            
        # if 'CV' in extra['name']:
        #     continue
            
        if 'Xi' in extra['name']:
            continue
                
        df = pd.DataFrame(chain, columns=model.get_labels())
    
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp
        
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extraname = ''
        
        # if i == 0:
        #     extraname = ' None'
        # elif i == 1:
        #     extraname = ' poly'
#         elif i == 2:
#             extraname = ' poly'
            
#         if 'CV' in extra['name']:
#             extraname = extraname + ' CV'
        if i == 1: 
            extraname = ' beta(Neff)'
            
        extra.pop("realisation", 'mean' + extraname)
        extra["name"] = extra['name'] + extraname
       
        c.add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        
        # model.set_data(data)
        # r_s = model.camb.get_data()["r_s"]
        # max_post = posterior.argmax()
        # params = df.loc[max_post]
        # params_dict = model.get_param_dict(chain[max_post])
        # for name, val in params_dict.items():
        #     model.set_default(name, val)

        # new_chi_squared, dof, bband, mods, smooths = model.plot(params_dict, display=False)
        # k = model.data[0]["ks"]
        # axes[i][count].scatter(k, (model.data[0]['pk0'][0]- smooths[0][0])*k, label= extraname + ' data')
        # axes[i][count].plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # axes[i][count].legend()
        
        #plt.scatter(k, (model.data[0]['pk0'][0]- smooths[0][0])*k, label= extraname + ' data')
        # plt.plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # plt.legend()
        # count += 1

        
        # if i == 0:
        #     default = smooths[0][0]
        # # plt.plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # plt.plot(k, (smooths[0][0]/default)*k, label=extraname)
        
        
    
# alpha_para, alpha_perp, beta = 0.97912566, 0.98881584, 1.0

alpha_para, alpha_perp = 1.00093140, 1.01083738
couplingterm = 1.0/0.22710731766023898
beta = (3.044/(3.044+couplingterm)) / (3.7/(3.7+couplingterm))  

alpha, eps = model.get_reverse_alphas(alpha_para, alpha_perp)
   
truth = {"$\\alpha$": alpha, "$\\epsilon$": eps, "$\\alpha_\\parallel$": alpha_para, "$\\alpha_\\perp$": alpha_perp, 
        "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": beta}

parameters = [names for names in truth]

# plt.legend()
# plt.xlabel(r'$k$ $\mathrm{Mpc}^{-1} h$')
# plt.ylabel(r'$k P_0(k)$ $\mathrm{Mpc}^2 h^{-2}$')

c.plotter.plot(
    truth=truth,
        parameters = parameters[:2]+parameters[4:], 
        legend=True,
        display=False,
        figsize=(11, 11), 
)

# some initial results with secondgen mocks

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 


path_p1 = '/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/'

# ELGs z = 0.8 - 1.1 
pfn1 = path_p1 + "desi_kp4_abacus_cubic_ELG-secondgenmocks/output/desi_kp4_abacus_cubic_ELG-secondgenmocks.fitter.pkl"
pfn2 = path_p1 + "desi_kp4_abacus_cubic_ELG-secondgenmocks-phaseshift/output/desi_kp4_abacus_cubic_ELG-secondgenmocks-phaseshift.fitter.pkl"

# ELGs z = 1.1 - 1.6 
pfn3 = path_p1 + "desi_kp4_abacus_cubic_ELG-secondgenmocks-z11-16/output/desi_kp4_abacus_cubic_ELG-secondgenmocks-z11-16.fitter.pkl"
pfn4 = path_p1 + "desi_kp4_abacus_cubic_ELG-secondgenmocks-phaseshift-z11-16/output/desi_kp4_abacus_cubic_ELG-secondgenmocks-phaseshift-z11-16.fitter.pkl"

# LRGs z = 0.4 - 0.6 
pfn5 = path_p1 + "desi_kp4_abacus_cubic_LRG-secondgenmocks-z04-06/output/desi_kp4_abacus_cubic_LRG-secondgenmocks-z04-06.fitter.pkl"
pfn6 = path_p1 + "desi_kp4_abacus_cubic_LRG-secondgenmocks-phaseshift-z04-06/output/desi_kp4_abacus_cubic_LRG-secondgenmocks-phaseshift-z04-06.fitter.pkl"

# LRGs z = 0.6 - 0.8 
pfn7 = path_p1 + "desi_kp4_abacus_cubic_LRG-secondgenmocks-z06-08/output/desi_kp4_abacus_cubic_LRG-secondgenmocks-z06-08.fitter.pkl"
pfn8 = path_p1 + "desi_kp4_abacus_cubic_LRG-secondgenmocks-phaseshift-z06-08/output/desi_kp4_abacus_cubic_LRG-secondgenmocks-phaseshift-z06-08.fitter.pkl"

# LRGs z = 0.8 - 1.1 
pfn9 = path_p1 + "desi_kp4_abacus_cubic_LRG-secondgenmocks-z08-11/output/desi_kp4_abacus_cubic_LRG-secondgenmocks-z08-11.fitter.pkl"
pfn10 = path_p1 + "desi_kp4_abacus_cubic_LRG-secondgenmocks-phaseshift-z08-11/output/desi_kp4_abacus_cubic_LRG-secondgenmocks-phaseshift-z08-11.fitter.pkl"

# QSOs z = 0.8 - 2.1 
pfn11 = path_p1 + "desi_kp4_abacus_cubic_QSO-secondgenmocks-z08-21/output/desi_kp4_abacus_cubic_QSO-secondgenmocks-z08-21.fitter.pkl"
pfn12 = path_p1 + "desi_kp4_abacus_cubic_QSO-secondgenmocks-phaseshift-z08-21/output/desi_kp4_abacus_cubic_QSO-secondgenmocks-phaseshift-z08-21.fitter.pkl"

# names = ['ELG z = 0.8 - 1.1', 'ELG z = 1.1 - 1.6', 'LRG z = 0.4 - 0.6', 
 #         'LRG z = 0.6 - 0.8', 'LRG z = 0.8 - 1.1', 'QSO z = 0.8 - 2.1']

pfns = [
    
        # pfn1, 
         # pfn2,
        # pfn3,
        # pfn4,
        # pfn5, 
        # pfn6,
        # pfn7, 
        #  pfn8,
        # pfn9, 
        # pfn10,
        pfn11, 
        pfn12,
    
]


c = ChainConsumer() 

for i,p in enumerate(pfns): 
    #print(i)
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    count = 0
    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']: # making sure we only get results from recon 
            continue  
                
        df = pd.DataFrame(chain, columns=model.get_labels())
    
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp
        
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extraname = ''

        if i%2 == 1: 
            extraname = ' beta(Neff)'
            
        extra.pop("realisation", 'mean' + extraname)
        # extra["name"] = names[i] + extraname
        extra["name"] = extra['name'] + extraname
        
       
        c.add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        
        # model.set_data(data)
        # r_s = model.camb.get_data()["r_s"]
        # max_post = posterior.argmax()
        # params = df.loc[max_post]
        # params_dict = model.get_param_dict(chain[max_post])
        # for name, val in params_dict.items():
        #     model.set_default(name, val)

        # new_chi_squared, dof, bband, mods, smooths = model.plot(params_dict, display=False)
        # k = model.data[0]["ks"]
        # axes[i][count].scatter(k, (model.data[0]['pk0'][0]- smooths[0][0])*k, label= extraname + ' data')
        # axes[i][count].plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # axes[i][count].legend()
        
        #plt.scatter(k, (model.data[0]['pk0'][0]- smooths[0][0])*k, label= extraname + ' data')
        # plt.plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # plt.legend()
        # count += 1

        
        # if i == 0:
        #     default = smooths[0][0]
        # # plt.plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # plt.plot(k, (smooths[0][0]/default)*k, label=extraname)
        
        
    
# alpha_para, alpha_perp, beta = 0.97912566, 0.98881584, 1.0

alpha_para, alpha_perp = 1.0, 1.0 # 1.00093140, 1.01083738
#couplingterm = 1.0/0.22710731766023898
#beta = (3.044/(3.044+couplingterm)) / (3.7/(3.7+couplingterm))  
beta = 1.0

alpha, eps = 1.0, 0.0 #model.get_reverse_alphas(alpha_para, alpha_perp)
   
truth = {"$\\alpha$": alpha, "$\\epsilon$": eps, "$\\alpha_\\parallel$": alpha_para, "$\\alpha_\\perp$": alpha_perp, 
        "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": beta}

parameters = [names for names in truth]

# plt.legend()
# plt.xlabel(r'$k$ $\mathrm{Mpc}^{-1} h$')
# plt.ylabel(r'$k P_0(k)$ $\mathrm{Mpc}^2 h^{-2}$')

c.plotter.plot(
    truth=truth,
        parameters = parameters[:2]+parameters[4:], 
        legend=True,
        display=False,
        figsize=(13, 13), 
)

# We want to compare the value of the fitted alpha for some results in which beta is and isn't free 

In [ ]:
# Import some necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

filepath_ELG_anisotropic_pk_recon = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-c003-broadbandpoly/output/Barry_fit_Pk_Recon.txt"
filepath_ELG_anisotropic_with_betaneff_pk_recon = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG_phaseshiftfree-c003-broadbandpoly/output/Barry_fit_Pk_Recon.txt" 

dat_anisotropic_pk_recon = pd.read_csv(filepath_ELG_anisotropic_pk_recon, skiprows=0, nrows=6, index_col=False)
dat_anisotropic_beta_pk_recon = pd.read_csv(filepath_ELG_anisotropic_with_betaneff_pk_recon, skiprows=0, nrows=6, index_col=False)
dat_anisotropic_pk_recon.columns = [i.strip() for i in dat_anisotropic_pk_recon.columns]
dat_anisotropic_beta_pk_recon.columns = [i.strip() for i in dat_anisotropic_beta_pk_recon.columns]

cm = matplotlib.colormaps['rainbow']
colors = dat_anisotropic_beta_pk_recon['beta'].to_numpy()
norm = mpl.colors.Normalize(vmin=np.min(colors), vmax=np.max(colors))

# add alpha, epsilon as additional rows to the models 
dat_anisotropic_pk_recon['alpha'] = dat_anisotropic_pk_recon['alpha_perp']**(1.0/3.0) * dat_anisotropic_pk_recon['alpha_par']**(2.0/3.0)
dat_anisotropic_pk_recon['epsilon'] = (dat_anisotropic_pk_recon['alpha_perp']/dat_anisotropic_pk_recon['alpha_par'])**(1.0/3.0) - 1.0
dat_anisotropic_beta_pk_recon['alpha'] = dat_anisotropic_beta_pk_recon['alpha_perp']**(1.0/3.0) * dat_anisotropic_beta_pk_recon['alpha_par']**(2.0/3.0)
dat_anisotropic_beta_pk_recon['epsilon'] = (dat_anisotropic_beta_pk_recon['alpha_perp']/dat_anisotropic_beta_pk_recon['alpha_par'])**(1.0/3.0) - 1.0

# plot the values against each other 
plt.figure(figsize=(16,7))
f, (ax1, ax2) = plt.subplots(1,2)

axes = [ax1, ax2] 

plt.rcParams['font.size'] = 12

ax1.plot([0.97, 1.03],[0.97, 1.03], linestyle='--', color='k')
ax1.scatter(dat_anisotropic_pk_recon['alpha'], dat_anisotropic_beta_pk_recon['alpha'], c=colors, cmap=cm)
ax1.set_xlabel(r"$\alpha$ standard")
ax1.set_ylabel(r"$\alpha$ with $\beta_{N_{\mathrm{eff}}}$")
ax1.set_title('First gen mocks, ELGs, c000, grid 000, template 000')
#ax1.set_xlim([0.97, 1.03])
#ax1.set_ylim([0.97, 1.03])

# plot the values against each other 
ax2.plot([-0.015, 0.015],[-0.015, 0.015], linestyle='--', color='k')
ax2.scatter(dat_anisotropic_pk_recon['epsilon'], dat_anisotropic_beta_pk_recon['epsilon'], c=colors, cmap=cm)
ax2.set_xlabel(r"$\epsilon$ standard")
ax2.set_ylabel(r"$\epsilon$ with $\beta_{N_{\mathrm{eff}}}$")
#plt.xlim([-0.01, 0.01])
#plt.ylim([-0.01, 0.01])
sm = plt.cm.ScalarMappable(cmap=cm, norm=norm)
#plt.colorbar(sm)
plt.colorbar(sm, label=r'$\beta_{N_{\mathrm{eff}}}$',ax=ax2)#, norm=norm)
plt.colorbar(sm, label=r'$\beta_{N_{\mathrm{eff}}}$',ax=ax1)#, norm=norm)

#sm.set_array([])
plt.subplots_adjust(left=0, right=2)
plt.show()



# comparing fit to best model fits based on settings 

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 
from chainconsumer import Chain, Truth 

pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-modelfitdata_spline/output/desi_kp4_abacus_cubic_ELG-modelfitdata_spline.fitter.pkl"

pfn2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-modelfitdata_poly/output/desi_kp4_abacus_cubic_ELG-modelfitdata_poly.fitter.pkl"
    
pfns = [
        pfn1, 
        pfn2
]

filelocs = [
    "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-modelfitdata_spline/output/desi_kp4_abacus_cubic_ELG-modelfitdata_spline-",
    "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-modelfitdata_poly/output/desi_kp4_abacus_cubic_ELG-modelfitdata_poly-"
]
plt.figure(figsize=(20,5))
c = ChainConsumer() 

for i,p in enumerate(pfns): 
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        print(extra['name'])
        if 'mean' in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']:
            continue 
            
        if '0' in extra['name']:
            #continue 
            namedat = 'best model fit \n from poly method'
            # count+= 1
        else:
            # continue 
            namedat = 'best model fit \n from spline method'
        if 'CV' in extra['name']:
            namedat = namedat + ' CV'
        
        df = pd.DataFrame(chain, columns=model.get_labels())
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp

        extraname = ''
        if i == 0:
            # continue 
            extraname = 'spline fit'
        if i == 1:  
            extraname = 'poly fit'
            
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        #extra.pop("realisation", namedat)
        #extra["name"] = extra['name'] + ', data is ' + namedat
        #print(df)
        #c.add_chain(Chain(samples=df[["$\\alpha$", "$\\epsilon$", "$\\beta_{\\phi(N_{\\mathrm{eff}})}$"]], name=extraname + ', data - ' + namedat + ' mock'))#, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        c.add_chain(Chain(samples=df, name=extraname + ', data - ' + namedat + ' mock'))#, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        
        df_for_file = df
        df['weights'] = weight
        df_for_file.to_csv(filelocs[i] + extra['name'] +'.csv')
        
truth = {"$\\alpha$": 1.0, "$\\epsilon$": 0.0, "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": 1.0}#, "$\\Sigma_s$": 0.0}# , "$\\Sigma_{nl,\\perp}$": 1.5, "$\\Sigma_{nl,||}$": 5.4
       # }#, "$\\alpha_\\perp$": 1.0, "$\\alpha_\\parallel$": 1.0}


mocksandmodels = {
            "poly best fit mean of mocks": [1.0007, 1.0005, 1.057],
            "poly best fit mean of mocks (CV)": [0.9977, 0.9975, 0.7452],
            "spline best fit mean of mocks": [1.0063, 1.0054, 1.4633],
            "spline best fit mean of mocks (CV)": [1.0018, 1.0013, 1.0549],
            "poly default": [1.0, 1.0, 1.0], 
            "spline default": [1.0, 1.0, 1.0] 
            }

alpha, eps = model.get_reverse_alphas(1.0007, 1.0005)
truthpoly = {"$\\alpha$": alpha, "$\\epsilon$": eps, "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": 1.057, 
            "$\\Sigma_s$": 0.0, "$\\Sigma_{nl,\\perp}$": 1.5, "$\\Sigma_{nl,||}$": 5.4}

alpha, eps = model.get_reverse_alphas(0.9977, 0.9975)
truthCVpoly = {"$\\alpha$": alpha, "$\\epsilon$": eps, "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": 0.7452,
               "$\\Sigma_s$": 0.0, "$\\Sigma_{nl,\\perp}$": 1.5, "$\\Sigma_{nl,||}$": 5.4}

alpha, eps = model.get_reverse_alphas(1.0063, 1.0054)
truthspline = {"$\\alpha$": alpha, "$\\epsilon$": eps, "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": 1.4633,
               "$\\Sigma_s$": 0.0, "$\\Sigma_{nl,\\perp}$": 1.5, "$\\Sigma_{nl,||}$": 5.4}

alpha, eps = model.get_reverse_alphas(1.0018, 1.0013)
truthCVspline = {"$\\alpha$": alpha, "$\\epsilon$": eps, "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": 1.0549,
                 "$\\Sigma_s$": 0.0, "$\\Sigma_{nl,\\perp}$": 1.5, "$\\Sigma_{nl,||}$": 5.4}

# c.add_truth(Truth(location=truthspline, color='orange', name='truth for spline model'))
# c.add_truth(Truth(location=truthCVspline, color='black', name='truth for CV spline model'))

#c.add_truth(Truth(location=truthpoly, color='orange', name='truth for poly model'))
#c.add_truth(Truth(location=truthCVpoly, color='black', name='truth for CV poly model'))

c.plotter.plot(columns=["$\\alpha$", "$\\epsilon$", "$\\beta_{\\phi(N_{\\mathrm{eff}})}$"])#,
                     #  "$\\Sigma_s$", "$\\Sigma_{nl,\\perp}$", "$\\Sigma_{nl,||}$"])

# looking at fits with 000 fake data 

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 
from chainconsumer import Chain, Truth 

pfn1 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-fitperfect000data_spline_phaseshift/output/desi_kp4_abacus_cubic_ELG-fitperfect000data_spline_phaseshift.fitter.pkl"
pfn2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-fitperfect000data_spline/output/desi_kp4_abacus_cubic_ELG-fitperfect000data_spline.fitter.pkl"

pfn3 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-fitperfect000data_poly_phaseshift/output/desi_kp4_abacus_cubic_ELG-fitperfect000data_poly_phaseshift.fitter.pkl"
pfn4 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-fitperfect000data_poly/output/desi_kp4_abacus_cubic_ELG-fitperfect000data_poly.fitter.pkl"
    
pfn5 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-fitperfect000data_poly-bestmarg/output/desi_kp4_abacus_cubic_ELG-fitperfect000data_poly-bestmarg.fitter.pkl"
pfn6 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-fitperfect000data_spline-bestmarg/output/desi_kp4_abacus_cubic_ELG-fitperfect000data_spline-bestmarg.fitter.pkl"

pfns = [
        pfn1, 
        pfn2,
        pfn3,
        pfn4,
        pfn5,
        pfn6
]

filelocs = [
       "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-fitperfect000data_spline_phaseshift/output/desi_kp4_abacus_cubic_ELG-fitperfect000data_spline_phaseshift-",
        "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-fitperfect000data_spline/output/desi_kp4_abacus_cubic_ELG-fitperfect000data_spline-",

        "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-fitperfect000data_poly_phaseshift/output/desi_kp4_abacus_cubic_ELG-fitperfect000data_poly_phaseshift-",
        "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-fitperfect000data_poly/output/desi_kp4_abacus_cubic_ELG-fitperfect000data_poly-",

        "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-fitperfect000data_poly-bestmarg/output/desi_kp4_abacus_cubic_ELG-fitperfect000data_poly-bestmarg-",
        "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG-fitperfect000data_spline-bestmarg/output/desi_kp4_abacus_cubic_ELG-fitperfect000data_spline-bestmarg-"
]

plt.figure(figsize=(20,5))
c = ChainConsumer() 

for i,p in enumerate(pfns): 
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)
    
    # if i < 4:
    #     continue 
    
#     if i == 1 or i == 3:
#         continue 

    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        print(extra['name'])
        
        if 'mean' in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' in extra['name']:
            continue 
        
        if 'realisation 0' in extra['name']:
            #continue 
            namedat = '000 model \n marg nuisance from poly method'
            #count+= 1
        else:
            # continue 
            namedat = '000 model fit \n marg nuisance from spline method'
        if 'CV' not in extra['name']:
            continue 
        else:
            namedat = namedat + ' CV'
        
        df = pd.DataFrame(chain, columns=model.get_labels())
        # Compute alpha_par and alpha_perp for each point in the chain
        # alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        # df["$\\alpha_\\parallel$"] = alpha_par
        # df["$\\alpha_\\perp$"] = alpha_perp

        extraname = ''
        if i == 0:
            extraname = 'spline fit phaseshift'
        if i == 1:  
            extraname = 'spline fit'
        if i == 2: 
            extraname = 'poly fit phaseshift'
        if i == 3:
            extraname = 'poly fit'
        if i == 4:
            extraname = 'poly fit marg only'
        if i == 5: 
            extraname = 'spline fit marg only'
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        #extra.pop("realisation", namedat)
        #extra["name"] = extra['name'] + ', data is ' + namedat
        # print(df.columns)
        c.add_chain(Chain(samples=df, name=extraname + ', data - ' + namedat))#, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
        
        df_for_file = df
        df['weights'] = weight
        df_for_file.to_csv(filelocs[i] + extra['name'] +'.csv')
        
        
truth = {"$\\alpha$": 1.0, "$\\epsilon$": 0.0, "$\\beta_{\\phi(N_{\\mathrm{eff}})}$": 1.0,
        "$b_{0,1}$": 0.52, '$\\beta$': 0.7, '$\\Sigma_s$': 0.0, '$\\Sigma_{nl,\\perp}$': 1.5, '$\\Sigma_{nl,||}$': 5.4}

#, "$\\alpha_\\perp$": 1.0, "$\\alpha_\\parallel$": 1.0}



c.add_truth(Truth(location=truth, color='black', name='truth'))

c.plotter.plot()#columns=["$\\alpha$", "$\\epsilon$", "$\\beta_{\\phi(N_{\\mathrm{eff}})}$"])